In [72]:
path = r"C:\Users\NGUYEN PHUONG BICH\HOC_TAP\NLP\Data\sentiments.csv"


In [73]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("TextClassificationExample") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()


In [74]:
df = spark.read.option("header", True).csv(path)
df.show(5)
df.printSchema


+--------------------+---------+
|                text|sentiment|
+--------------------+---------+
|Kickers on my wat...|        1|
|user: AAP MOVIE. ...|        1|
|user I'd be afrai...|        1|
|   MNTA Over 12.00  |        1|
|    OI  Over 21.37  |        1|
+--------------------+---------+
only showing top 5 rows


<bound method DataFrame.printSchema of DataFrame[text: string, sentiment: string]>

In [75]:
df.groupBy("sentiment").count().show()

+---------+-----+
|sentiment|count|
+---------+-----+
|       -1| 2106|
|     NULL|    1|
|        1| 3685|
+---------+-----+



In [76]:
ff = df.dropna()
ff.count()

5791

In [77]:
from pyspark.ml.feature import *

indexer = StringIndexer().setInputCol("sentiment").setOutputCol("label")
tokenizer = RegexTokenizer().setInputCol('text').setOutputCol('tokens').setPattern("""[\s:.?!;'*]+""")
vectorizer = CountVectorizer(vocabSize=10000).setInputCol('tokens').setOutputCol("features")

In [78]:
from pyspark.ml import *

pipeline = Pipeline().setStages([indexer, tokenizer, vectorizer])
model = pipeline.fit(ff)
gf = model.transform(ff)
gf.show(5)

+--------------------+---------+-----+--------------------+--------------------+
|                text|sentiment|label|              tokens|            features|
+--------------------+---------+-----+--------------------+--------------------+
|Kickers on my wat...|        1|  0.0|[kickers, on, my,...|(10000,[3,44,69,7...|
|user: AAP MOVIE. ...|        1|  0.0|[user, aap, movie...|(10000,[0,6,7,13,...|
|user I'd be afrai...|        1|  0.0|[user, i, d, be, ...|(10000,[1,2,4,8,1...|
|   MNTA Over 12.00  |        1|  0.0|[mnta, over, 12, 00]|(10000,[24,124,14...|
|    OI  Over 21.37  |        1|  0.0|  [oi, over, 21, 37]|(10000,[24,369,40...|
+--------------------+---------+-----+--------------------+--------------------+
only showing top 5 rows


In [79]:
from pyspark.ml.classification import *

classifier = LogisticRegression().setFeaturesCol("features").setLabelCol("label")
pipeline = Pipeline().setStages([indexer, tokenizer, vectorizer, classifier])

In [80]:
model = pipeline.fit(ff)
output = model.transform(ff)
output.show(5)
output.select("label", "prediction").show(30)

+--------------------+---------+-----+--------------------+--------------------+--------------------+--------------------+----------+
|                text|sentiment|label|              tokens|            features|       rawPrediction|         probability|prediction|
+--------------------+---------+-----+--------------------+--------------------+--------------------+--------------------+----------+
|Kickers on my wat...|        1|  0.0|[kickers, on, my,...|(10000,[3,44,69,7...|[26.5136148403175...|[0.99999999999694...|       0.0|
|user: AAP MOVIE. ...|        1|  0.0|[user, aap, movie...|(10000,[0,6,7,13,...|[38.3455254608298...|           [1.0,0.0]|       0.0|
|user I'd be afrai...|        1|  0.0|[user, i, d, be, ...|(10000,[1,2,4,8,1...|[30.4722937408601...|[0.99999999999994...|       0.0|
|   MNTA Over 12.00  |        1|  0.0|[mnta, over, 12, 00]|(10000,[24,124,14...|[14.2768538363632...|[0.99999936956417...|       0.0|
|    OI  Over 21.37  |        1|  0.0|  [oi, over, 21, 37]|(10

In [81]:
from pyspark.ml.evaluation import *

evaluator = BinaryClassificationEvaluator()
evaluator.evaluate(output)

0.9999994201486739

In [82]:
train, test = ff.randomSplit([0.8, 0.2], seed=123)

In [84]:
model = pipeline.fit(train)
testOutput = model.transform(test)
evaluator.evaluate(testOutput)

0.8075973391222627

In [ ]:
train, test = ff.randomSplit([0.8, 0.2], seed=123)

from pyspark.ml.classification import *
classifier = MultilayerPerceptronClassifier().setLayers([1000, 50, 2])
pipeline = Pipeline().setStages([indexer, tokenizer, vectorizer, classifier])
model = pipeline.fit(train)

In [111]:
testOutput = model.transform(test)
evaluator.evaluate(testOutput)

Py4JJavaError: An error occurred while calling o4545.evaluate.
: org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] User defined function (`ProbabilisticClassificationModel$$Lambda$4973/0x0000000802412df0`: (struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) failed due to: java.lang.IllegalArgumentException: requirement failed: A & B Dimension mismatch!. SQLSTATE: 39000
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:195)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenEvaluatorFactor

In [1]:
from datasets import load_dataset

ds = load_dataset("zeroshot/twitter-financial-news-sentiment")

c:\Users\NGUYEN PHUONG BICH\HOC_TAP\NLP\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import pandas as pd
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

train_df = spark.createDataFrame(pd.DataFrame(ds['train'])[:1000])
validation_df = spark.createDataFrame(pd.DataFrame(ds['validation']))

In [6]:
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 9543
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2388
    })
})

In [5]:
train_df.printSchema()
train_df.show(3)

root
 |-- text: string (nullable = true)
 |-- label: long (nullable = true)



Py4JJavaError: An error occurred while calling o104.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1.0 (TID 1) (10.0.39.205 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:252)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:143)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:158)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:178)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:261)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:70)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:374)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:338)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartition

In [8]:
import os
import findspark

# 🔧 Khai báo đường dẫn Python thật trong môi trường ảo
os.environ["PYSPARK_PYTHON"] = os.path.join(os.getcwd(), ".venv", "Scripts", "python.exe")
os.environ["PYSPARK_DRIVER_PYTHON"] = os.path.join(os.getcwd(), ".venv", "Scripts", "python.exe")

# Khởi tạo Spark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("TwitterSentiment") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.python.worker.reuse", "true") \
    .config("spark.network.timeout", "600s") \
    .config("spark.executor.heartbeatInterval", "60s") \
    .getOrCreate()

print("✅ Spark started with version:", spark.version)


✅ Spark started with version: 4.0.1


In [9]:
data = [("Hello world", 1), ("Bad day", 0), ("Profits rose", 1)]
df = spark.createDataFrame(data, ["text", "label"])
df.show()


Py4JJavaError: An error occurred while calling o156.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 2) (10.0.39.205 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:252)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:143)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:158)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:178)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:261)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:70)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:374)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:338)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartition

In [4]:
import os
import sys
import re

from abc import ABC, abstractmethod
from typing import List

class Tokenizer(ABC):
    
    def tokenize(self, text:str) -> List:

        '''Tách một chuỗi đầu vào thành 1 list các token'''
        pass

class Vectorizer(ABC):

    def fit(self, corpus: list[str]):
        '''
        Learns the vocabulary from a list of ducuments (corpus).
        '''
        pass

    def transform(self, documents: list[str]) -> list[list[int]]:
        '''
        Transform a list of documents into a list of count vectors based on the learned vocabulary.
        '''
        pass

    def fit_transform(self, corpus: list[str]) -> list[list[int]]:
        '''
        A convernience method that performs fit and then transform on the same data.
        '''
        pass

In [ ]:
class RegexTokenizer(Tokenizer):
    def tokenize(self, text: str):
        tokens = re.findall(r'\w+|[^\w\s]', text, re.UNICODE)
        return tokens

class CountVectorizer(Vectorizer):
    def __init__(self, tokenizer: Tokenizer):
        self.tokenizer = tokenizer
        self.vocabulary_ = dict[str, int]()
    
    def fit(self, corpus: list[str]):
        '''
        - Khởi tạo một tập hợp rỗng để giữ các token duy nhất.
        - Lặp qua từng tài liệu trong kho dữ liệu.
        - Với mỗi tài liệu, sử dụng tokenizer đã được cung cấp để lấy ra danh sách các token.
        - Thêm tất cả các token vào tập hợp có được một từ vựng duy nhất.
        - Sau khi đã xử lí hết tài liệu, tạo một từ điển vocabulary_ ánh xạ mỗi token với một chỉ số nguyên duy nhất.
        '''
        vocab_set = set()
        for document in corpus:
            tokens = self.tokenizer.tokenize(document)
            vocab_set.update(tokens)
        self.vocabulary_ = {word: idx for idx, word in enumerate(sorted(vocab_set))}

    def transform(self, documents: list[str]) -> list[list[int]]:
        '''
        - Với mỗi tài liệu trong danh sách đầu vào:
            - Tạo một vector 0 có độ dài bằng với kích thước của từ điển vocabulary_.
            - Với mỗi token, nếu tồn tại trong vocabulary_ thì tăng số đếm tại chỉ số tương ứng trong vector 0.
        - Trả về danh sách các vector kết quả.
        '''
        if not self.vocabulary_:
            raise ValueError("Vocabulary is empty. Please fit the vectorizer before transforming data.")
        
        vectors = []
        for document in documents:
            tokens = self.tokenizer.tokenize(document)
            vector = [0] * len(self.vocabulary_)
            for token in tokens:
                if token in self.vocabulary_:
                    index = self.vocabulary_[token]
                    vector[index] += 1
            vectors.append(vector)
        return vectors
    
    def fit_transform(self, corpus: list[str]) -> list[list[int]]:
        self.fit(corpus)
        return self.vocabulary_, self.transform(corpus)

In [ ]:
text = {
    "This movie is fanstatic and I love it!",
    "I hate this film, it's terrible",
    "The acting was superb, a truly great experience.",
    "What a waste of time, absolutely boring.",
    "Highly a recommend this, a masterpiece.",
    "Could not finish watching, so bad."
}

labels = [1,0,1,0,1,0] # 1 - positive, 0 - negative

In [ ]:
tokenizers = []

regexTokenizer = RegexTokenizer()

for t in text:
    tokenizers.append(regexTokenizer.tokenize(t))

In [18]:
count_vectorizer = CountVectorizer(tokenizer=RegexTokenizer())
vocabulary, vectors = count_vectorizer.fit_transform(list(text))


In [15]:
print(vocabulary)

{'!': 0, "'": 1, ',': 2, '.': 3, 'Could': 4, 'Highly': 5, 'I': 6, 'The': 7, 'This': 8, 'What': 9, 'a': 10, 'absolutely': 11, 'acting': 12, 'and': 13, 'bad': 14, 'boring': 15, 'experience': 16, 'fanstatic': 17, 'film': 18, 'finish': 19, 'great': 20, 'hate': 21, 'is': 22, 'it': 23, 'love': 24, 'masterpiece': 25, 'movie': 26, 'not': 27, 'of': 28, 'recommend': 29, 's': 30, 'so': 31, 'superb': 32, 'terrible': 33, 'this': 34, 'time': 35, 'truly': 36, 'was': 37, 'waste': 38, 'watching': 39}


In [20]:
len(vectors)

6

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *
from typing import Dict

class TextClassifier:
    def __init__(self, vectorizer: Vectorizer):
        self.vectorizer = vectorizer
        self._model = LogisticRegression(solver='liblinear')

    def fit(self, texts:List[str], labels:List[int]):
        vocabulary, X = self.vectorizer.fit_transform(list(texts))
        self._model.fit(X, labels)

    def predict(self, texts: List[str]) -> List[int]:
        vectors = self.vectorizer.transform(texts)
        return self._model.predict(vectors)

    def evaluate(self, y_true: List[int], y_pred: List[int]) -> Dict[str, float]:
        return {
            "accuracy_score": accuracy_score(y_true, y_pred),
            "precision_score": precision_score(y_true, y_pred),
            "recall_score": recall_score(y_true, y_pred),
            "f1_score": f1_score(y_true, y_pred)
        }

In [33]:
from sklearn.model_selection import train_test_split
text = [
    "This movie is fanstatic and I love it!",
    "I hate this film, it's terrible",
    "The acting was superb, a truly great experience.",
    "What a waste of time, absolutely boring.",
    "Highly a recommend this, a masterpiece.",
    "Could not finish watching, so bad."
]

labels = [1,0,1,0,1,0] # 1 - positive, 0 - negative

text_train, text_test, label_train, label_test = train_test_split(text, labels, test_size=0.2, random_state=42)

count_vectorizer = CountVectorizer(tokenizer=RegexTokenizer())
text_classifier = TextClassifier(count_vectorizer)

text_classifier.fit(text_train, label_train)
label_pred = text_classifier.predict(text_test)
print(text_classifier.evaluate(label_test, label_pred))

{'accuracy_score': 0.0, 'precision_score': 0.0, 'recall_score': 0.0, 'f1_score': 0.0}


In [37]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("SentimentAnalysis").getOrCreate()

In [47]:
from pyspark.sql.column import Column
data_path = r'C:\Users\NGUYEN PHUONG BICH\HOC_TAP\NLP\Data\sentiments.csv'
df = spark.read.csv(data_path, header=True, inferSchema=True)
df = df.withColumn("label",(df.sentiment.cast("integer") + 1)/2)
initial_row_count = df.count()
df = df.dropna(subset=["sentiment"])

In [ ]:
from pyspark.ml.feature import *
from pyspark.ml import *

tokenizer = Tokenizer(inputCol="text", outputCol="words")
stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered_word")
hashingTF = HashingTF(inputCol="filtered_words", outputCol="raw_features", numFeatures=1)
idf = IDF(inputCol="raw_features",outputCol="features")

lr = LogisticRegression(C='features', max_iter=10, )
pipeline = Pipeline(stages=[tokenizer, stopwordsRemover, hashingTF, idf, lr])

In [56]:
size = df.count()
trainingData = df[:int(size*0.8)]
testingData = df[int(size*0.8):]

PySparkTypeError: [NOT_COLUMN_OR_FLOAT_OR_INT_OR_LIST_OR_STR] Argument `item` should be a Column, float, integer, list or string, got slice.

+--------------------+---------+-----+
|                text|sentiment|label|
+--------------------+---------+-----+
|Kickers on my wat...|        1|  1.0|
|user: AAP MOVIE. ...|        1|  1.0|
|user I'd be afrai...|        1|  1.0|
|   MNTA Over 12.00  |        1|  1.0|
|    OI  Over 21.37  |        1|  1.0|
|   PGNX  Over 3.04  |        1|  1.0|
|AAP - user if so ...|       -1|  0.0|
|Monday's relative...|       -1|  0.0|
|GOOG - ower trend...|        1|  1.0|
|AAP will watch to...|        1|  1.0|
|i'm assuming FCX ...|        1|  1.0|
|It really worries...|        1|  1.0|
|AAP GAMCO's arry ...|        1|  1.0|
|user Maykiljil po...|        1|  1.0|
|Momentum is comin...|        1|  1.0|
|HA Hitting 35.65 ...|        1|  1.0|
|user gameplan sho...|        1|  1.0|
|with FCX gapping ...|        1|  1.0|
|user great list a...|        1|  1.0|
|ATHX upper trend ...|        1|  1.0|
+--------------------+---------+-----+
only showing top 20 rows
